# Scrapy 입문
> 간단한 Scrapy 개념과 실습을 정리하였습니다

- toc: true
- badges: true
- comments: false
- categories: [crawling]
- image:

## Scrapy

> - 크롤링을 좀더 안정적으로 할 수 있음
>   - Scrapy 내부에서 다양한 안정장치가 있음
> - 크롤링을 좀더 빠르게 할 수 있음
>   - 크롤링 프로그램을 동시에 여러개 실행시켜서 많은 량의 데이터 크롤링시, 시간 단축
> - 다양한 크롤링 관련 기능
    - 크롤링한 데이터를 다양한 포멧으로 저장도 가능

## 사용 방법

1. 실제 크롤링할 스파이더(spider, scrapy 기반 크롤링 프로그램) 생성 <br>
2. 크롤링할 사이트(시작점)와 크롤링할 아이템(item)에 대한 selector 설정 <br>
3. 크롤러 실행

윈도우, 맥 공통
```shell
pip install scrapy
```

프로젝트 생성

1. 터미널 오픈
2. scrapy startproject <프로젝트 이름>
```shell
scrapy startproject ecommerce
```
3. 생성된 프로젝트 확인

``
cd ecommerce
``
<br>ecommerce 폴더 내의 파일 구조 확인
```shell

scrapy.cfg # deploy configuration file
ecommerce/ # project's Python module, you'll import your code from her
    __init__.py
    items.py # project items definition file
    pipelines.py # project pipelines file
    settings.py # project settings file
    spiders/ # a directory where you'll later put your spiders
        __init__.py


```

`scrapy -h` 명령어를 통해 사용법을 간략하게나마 알 수 있다.

- ecommerce/ecommerce 폴더에서 다음 명령으로 작성 가능
- ``genspider``을 통해 크롤러 생성 

```shell

scrapy genspider <크롤러이름> <크롤링페이지주소>
scrapy genspider gmarket www.gmarket.co.kr

// 해당 주소에는 https://를 붙여주지 않는 것이 좋다. 

>> Created spider 'gmarket' using template 'basic' in module: ecommerce.spiders.gmarket

```
 __-> ecommerce/ecommerce/spiders 디렉토리에 gmarket.py 파일(템플릿)이 생김__


#### scrapy template 설명

- gmarket.py

In [2]:
import scrapy

# scrapy.Spider을 상속해서 spider의 메소드를 사용 
class GmarketSpider(scrapy.Spider):
    name = 'gmarket'
    allowed_domains = ['www.gmarket.co.kr'] # 특정 도메인만 허용: 다른 사이트의 크롤링을 방지 
    start_urls = ['http://www.gmarket.co.kr/']

    def parse(self, response):
        print(response.text) # 이 부분을 수정해준다
        


```shell
scrapy crawl gmarket 
```
위의 명령문으로 크롤링을 실행, 홈페이지 텍스트를 전부 가져오기 때문에 엄청 복잡한 정보가 나온다

- 위의 gmarket.py를 통하여 기본 템플릿 구조를 알 수 있다.

    - `start_urls` 가 중요한데, 이는 `크롤링할 페이지 주소`를 나타내기 때문이다.
        - start_urls는 리스트로 크롤링할 주소를 여러개 써도 된다.
    - parse 함수는 클래스의 메서드로 response를 반드시 인자로 받아야 한다
        - response에 start_urls 에 기록된 주소의 크롤링 결과가 담아져 온다.
    - response에는 크롤링된 데이터가 담겨져 있습니다
        - response.text에 크롤링된 데이터가 담겨져 있습니다. 
        
        

In [3]:
import scrapy

class GmarketSpider(scrapy.Spider):
    name = 'gmarket'
    allowed_domains = ['www.gmarket.co.kr'] # 특정 도메인만 허용: 다른 사이트의 크롤링을 방지
    start_urls = ['http://www.gmarket.co.kr/']

    def parse(self, response):
        print(response.url) # response의 url을 출력해준다



### scrapy shell 사용

``` shell
scrapy shell 'http://corners.gmarket.co.kr/Bestsellers'

```

해당 링크의 정보를 가져와서 명령을 기다리는 상태가 된다.

![]({{site.baseurl}}/images/scrapy/명령기다림.png)

- 명령어
```shell
view(response)
// 크롤링한 페이지를 웹브라우저를 통해 확인하기

response.url
// 크롤링한 페이지 주소 확인

```


- response.css()꞉ css selector 로 데이터 가져올 수 있다.
```shell
response.css('head > title').get()
response.css('head > title').getall() # 리스트 형태로 가져옴
response.css('head > title::text').get() # 태그가 없어진 채로 나온다
# Out[5]: 'G마켓 - G마켓 베스트'

response.css('div.best-list li > a::text').getall()
#  gmarket의 베스트 상품의 타이틀 텍스트를 가져올 수 있다
response.css('div.best-list li > a::text')[1].get() # 특정 리스트의 인덱스 순번에 따라서 가져옴

```

- response.xpath()꞉ xpath 로 데이터 가져오기
```python
div.best-list ul li a
xpath => div[@class='best-list']/ul/li/a

response.xpath('//div[@class="best-list"]/ul/li/a').getall() # 전부 가져오기
response.xpath('//div[@class="best-list"]/ul/li/a/text()').getall() # 텍스트만 가져오기

```


- 정규표현식 사용 가능

```python
response.css('div.best-list li > a::text')[1].get()
## Out[6]: '(무료반품) 쉬블림로브 12%쿠폰 블라우스 원피스 팬츠'

## ? 는 앞 문자가 0 ~ 1 번 표시되는 패턴
## * 는 앞 문자가 0 ~ 번 표시되는 패턴
## + 는 앞 문자가 1번 또는 그 이상 반복되는 패턴
response.css('div.best-list li > a::text')[1].re('(\w+)')
## Out[7]: ['무료반품', '쉬블림로브', '12', '쿠폰', '블라우스', '원피스', '팬츠']

# xpath 사용 가능
response.xpath('//div[@class="best-list"]/ul/li/a/text()')[1].re('(\w+)')
```